In [1]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import data_extraction
import ARIMA
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
nRowsRead = None
grid_number = 1
df = pd.read_pickle('forecast1.pkl')

In [3]:
def data_ANN(past_weeks, df):
    # input values
    # past_vals is the number of past values of the residuals that the ANN will use
    residuals = df["residuals"]

    X = []
    y = []
    
    for i in range(len(residuals.values) - past_weeks*24*7):
        X_el = residuals[i:i+24*7*past_weeks:24*7]
        X.append(X_el)
        y.append([residuals.iloc[i+past_weeks*24*7]])
    
    return X, y


In [4]:
def getgsCV(numberOfPastVals):
    X, y = data_ANN(numberOfPastVals, df)
    
    train_size_percentage = 0.50

    train_size = int(len(X)*train_size_percentage)

    X_train = X[:train_size]
    y_train = y[:train_size]

    X_test = X[train_size:]
    y_test = y[train_size:]
    
    model = MLPRegressor(solver="adam", 
                         alpha=1e-5, 
                         random_state=1, 
                         max_iter=20000,
                         tol=1e-4)
    parameters = {'solver': ['adam'], 'hidden_layer_sizes' : [(i) for i in range(1,numberOfPastVals*2,2)] + [(i,i) for i in range(1,round(6/2),2)] + [(i,i,i) for i in range(1,round(7/3),2)]}
    gsCV = GridSearchCV(model , parameters)
    gsCV.fit(X_train, y_train)
    return gsCV

In [5]:
gsCVList = []
past_vals = [i for i in range(4,5,1)]

In [6]:
for val in tqdm(past_vals):
    gsCVList.append(getgsCV(val))

100%|██████████| 1/1 [01:04<00:00, 64.39s/it]


In [56]:
CVdf = pd.DataFrame(index = past_vals)

bestScores = []
bestHiddenLayers = []

for i in range(len(gsCVList)):
    bestScores.append(gsCVList[i].best_score_)
    bestHiddenLayers.append(gsCVList[i].best_params_['hidden_layer_sizes'])

In [59]:
CVdf['past'] = past_vals
CVdf['scores'] = bestScores
CVdf['hidden layers'] = bestHiddenLayers

In [61]:
CVdf.to_pickle("hyper_parameters_hybrid.pkl")

In [63]:
pd.read_pickle("hyper_parameters_hybrid.pkl")

,scores,past,hidden layers
2,0.263702,2,"(4, 4)"
4,0.354967,4,"(4, 4)"


In [12]:
gsCVList[0].best_index_

0

In [11]:
gsCVList[0].best_score_

0.3395936743121642

In [15]:
gsCVList[0].cv_results_

{'mean_fit_time': array([4.79402804, 1.25299864, 2.15971551, 2.59400439, 0.37395697,
        0.39781837]),
 'std_fit_time': array([0.54143088, 0.08609949, 0.30166798, 0.48230313, 0.02796609,
        0.01631202]),
 'mean_score_time': array([0.0170547 , 0.01681538, 0.01592417, 0.01630621, 0.01647539,
        0.01612692]),
 'std_score_time': array([0.00083852, 0.00096178, 0.00043415, 0.00058275, 0.00088599,
        0.00045306]),
 'param_hidden_layer_sizes': masked_array(data=[1, 3, 5, 7, (1, 1), (1, 1, 1)],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_solver': masked_array(data=['adam', 'adam', 'adam', 'adam', 'adam', 'adam'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'hidden_layer_sizes': 1, 'solver': 'adam'},
  {'hidden_layer_sizes': 3, 'solver': 'adam'},
  {'hidden_layer_sizes': 5, 'solver': 'adam'},
  {'hidden_layer_sizes':

In [17]:
gsCVList[0].best_params_

{'hidden_layer_sizes': 1, 'solver': 'adam'}